# Cleaning Data
If you want to type along with me, use [this notebook](https://humboldt.cloudbank.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fbethanyj0%2Fdata271_sp24&branch=main&urlpath=tree%2Fdata271_sp24%2Fdemos%2Fdata271_demo28_live.ipynb) instead. 
If you don't want to type and want to follow along just by executing the cells, stay in this notebook. 

In [4]:
import numpy as np
import pandas as pd

In [ ]:
# import the data from last time
df = pd.read_csv('youtube_humboldt.csv')
df = df.iloc[:,1:]
df.head()

## Cleaning the data 

### Reformat column names

In [ ]:
# OPTION 1 .rename method
df.rename(columns = {'videoId':'video_id',
                                'publishedAt':'published_at',
                                'viewCount':'view_count',
                                'likeCount':'like_count'})

In [ ]:
# does not update original (use inline=True)
df

In [ ]:
# OPTION 2: more general approach that would work for many columns
# Split column names by words
import re
split_by_words = [re.split('(?=[A-Z])',i) for i in df.columns]
split_by_words

In [ ]:
# Insert an underscore between words
columns_with_underscores = ['_'.join(i) for i in split_by_words]
columns_with_underscores

In [ ]:
# Reassign column names
df.columns = columns_with_underscores

In [ ]:
# Make everything lowercase
df.columns = df.columns.str.lower()
df.head()

### Convert datatypes

In [ ]:
# Suppose we wanted to look at the relationship between view counts and like counts
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.scatterplot(data = df, x='view_count',y='like_count')
plt.show()

In [ ]:
# Something is funky. Check the data types
df.dtypes

In [ ]:
# convert the data types
df = df.assign(view_count = df.view_count.astype(int),
                like_count = df.like_count.astype(int))

In [ ]:
# check data types again
df.dtypes

In [ ]:
# plot again
sns.scatterplot(data = df, x='view_count',y='like_count')
plt.show()

In [ ]:
# What if we wanted to plot trend in view counts?
sns.lineplot(data = df, x='published_at',y='view_count')
plt.show()

In [ ]:
# change dates to datetime data
df = df.assign(date = pd.to_datetime(df.published_at))
df.head()

In [ ]:
# Check dtypes again
df.dtypes

In [ ]:
# plot again
sns.lineplot(data = df, x='date',y='view_count')
plt.show()

#### More on working with dates

In [ ]:
date1 = 'March 29, 2024'
pd.to_datetime(date1, format = '%B %d, %Y')

In [ ]:
date2 = 'Mar 29, 2024'
pd.to_datetime(date2, format = '%b %d, %Y')

In [ ]:
date3 = '3/29/10'
pd.to_datetime(date3, format = '%m/%d/%y')

In [ ]:
date4 = '29-03-10'
pd.to_datetime(date4, format = '%d-%m-%y')

### Reset the index

In [ ]:
# Reset the index for easy access by video id
df.set_index('video_id',inplace=True)
df.head()

In [ ]:
# index by video id
video_id = "GpOplrOC7X0"
df.loc[video_id]

In [ ]:
df.loc[df.view_count.idxmax()]

### Reorder the data

In [ ]:
# reorder rows
df.sort_index(axis=0)

In [ ]:
# doesn't update original
df

In [ ]:
# reorder columns
df.sort_index(axis=1)

In [ ]:
# reorder columns manually
df[['view_count','like_count','date','published_at','title']]

In [ ]:
# sort the data 
df.sort_values(by='view_count')

In [ ]:
# doesn't update original
df

In [ ]:
# do it inplace
df.sort_values(by = 'view_count',ascending=False, inplace=True)

In [ ]:
df

In [ ]:
# sort by multiple values at once
df.sort_values(by = ['view_count','like_count'],ascending=[False,True], inplace=True)
df

### Reshaping data

In [5]:
df_weather_wide = pd.read_csv('sample_weather.csv')
df_weather_wide = df_weather_wide.iloc[:,1:]
df_weather_wide

,date,inches_of_rain,max_temp,min_temp
0,2024-01-01,0.5,60,40
1,2024-01-02,0.0,55,45
2,2024-01-03,0.1,52,42
3,2022-01-04,0.0,56,48


In [6]:
# transpose the data
df_weather_wide.T

,0,1,2,3
date,2024-01-01,2024-01-02,2024-01-03,2022-01-04
inches_of_rain,0.5,0.0,0.1,0.0
max_temp,60,55,52,56
min_temp,40,45,42,48


In [ ]:
# transpose with more informative columns
df_weather_wide.set_index('date').T

In [7]:
# change wide format data into long format
long_df = df_weather_wide.melt(id_vars = 'date',value_vars = ['max_temp','min_temp','inches_of_rain'])
long_df

,date,variable,value
0,2024-01-01,max_temp,60.0
1,2024-01-02,max_temp,55.0
2,2024-01-03,max_temp,52.0
3,2022-01-04,max_temp,56.0
4,2024-01-01,min_temp,40.0
5,2024-01-02,min_temp,45.0
6,2024-01-03,min_temp,42.0
7,2022-01-04,min_temp,48.0
8,2024-01-01,inches_of_rain,0.5
9,2024-01-02,inches_of_rain,0.0


In [ ]:
# change long format back into wide format
long_df.pivot(index = 'date',columns = 'variable',values='value')

### What do when there are multiple values in categories 

In [ ]:
long_df = pd.read_csv('long_data.csv')
long_df = long_df.iloc[:,1:]
long_df.head()

In [ ]:
# Pivot the data to get average sales by date and category
long_df.pivot_table(index=['date'], columns='category', values=['sales'])

In [ ]:
# Pivot the data to get TOTAL sales by date and category
wide_df = long_df.pivot_table(index=['date'], columns='category', values=['sales'], aggfunc=sum)
wide_df

In [ ]:
# Pivot the data to get TOTAL sales by date, product, and category
long_df.pivot_table(index='date', columns=['category','product'], values=['sales'], aggfunc=sum)

In [ ]:
# Go from wide to long
wide_df.reset_index().melt(id_vars='date', var_name=['type','category'])

## Activity

In [ ]:

# Create a DataFrame with data cleaning and reshaping opportunities
data = {
    'Pet Name': ['Fluffy', 'Whiskers', 'Bubbles', 'Spike', 'Coco', 'Maybelle', 'Snowball'],
    'Date Adopted': ['10-01-2023','03-04-2024','01-10-2024','02-14-2024','11-22-2023','01-04-2024','12-25-2025'],
    'Animal Type': ['Cat', 'Cat', 'Fish', 'Dog', 'Fish', 'Dog', 'Cat'],
    'Pet Age': ['3', '2', '13', '5', '4', '3', '2'],
    'Color': ['White', 'Gray', 'Orange', 'White', 'White', 'Black', 'Black'],
    'Happiness Level': ['High', 'Medium', 'High', 'Low', 'High', 'High', 'Medium']
}
df_pets = pd.DataFrame(data)
df_pets

**Activity 1:** Rename the columns of the pets dataframe to be in a better format.

**Activity 2:** Change any datatypes that should be adjusted.  

**Activity 3:** Practice pivoting the dataframe.